In [5]:
import findspark
findspark.init('/home/ubuntu/spark-3.2.0-bin-hadoop3.2')
DATA_PATH = '../data/Spark_for_Machine_Learning/Logistic_Regression/'

In [2]:
from pyspark.sql import SparkSession

In [9]:
from pyspark.ml.classification import LogisticRegression

In [3]:
spark = SparkSession.builder.appName('myLogreg').getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/29 15:53:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Load data

In [16]:
data = spark.read.csv(DATA_PATH + 'titanic.csv', header=True, inferSchema=True)

In [18]:
data.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



## Preprocess

In [25]:
from pyspark.ml.feature import (VectorAssembler, 
                                VectorIndexer, 
                                OneHotEncoder, 
                                StringIndexer)

In [21]:
target = 'Survived'
raw_feats = [
 'Pclass',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'Embarked']
raw_cols = [target] + feats

In [22]:
df = data.select(raw_cols).na.drop()

In [26]:
# Encode string feats
gender_indexer = StringIndexer(inputCol='Sex', outputCol='SexIndex')
gender_encoder = OneHotEncoder(inputCol='SexIndex', outputCol='SexVec')

embark_indexer = StringIndexer(inputCol='Embarked',outputCol='EmbarkIndex')
embark_encoder = OneHotEncoder(inputCol='EmbarkIndex', outputCol='EmbarkVec')

In [33]:
proc_feats = ['Pclass','SexVec','Age','SibSp','Parch','Fare','EmbarkVec']
proc_cols = [target] + proc_feats
assembler = VectorAssembler(inputCols=proc_feats, outputCol='features')

## Make pipeline and fit

In [29]:
from pyspark.ml import Pipeline

In [30]:
lr = LogisticRegression(featuresCol='features', labelCol='Survived')

In [34]:
pipe = Pipeline(stages=[
    gender_indexer,
    embark_indexer,
    gender_encoder,
    embark_encoder,
    assembler,
    lr
])

In [35]:
train_df, test_df = df.randomSplit([0.7,0.3])

In [37]:
fit_pipe = pipe.fit(train_df)

In [38]:
results = fit_pipe.transform(test_df)

In [41]:
results.printSchema()

root
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Embarked: string (nullable = true)
 |-- SexIndex: double (nullable = false)
 |-- EmbarkIndex: double (nullable = false)
 |-- SexVec: vector (nullable = true)
 |-- EmbarkVec: vector (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



## Evaluate

In [39]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [55]:
bin_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='Survived',  metricName='areaUnderROC')

In [56]:
eval_results = bin_eval.evaluate(results)

In [51]:
# AUC
eval_results

0.8010168302945301